In [2]:
import numpy as np

def chunkify(board:np.array):
    H,W = board.shape
    sq = int(H ** 0.5)
    return board.reshape(sq,sq,sq,sq).swapaxes(1, 2).reshape(sq ** 2,sq ** 2)

def verify(board):
    for row in board:
        if True in (np.unique(row[row != None],return_counts=True)[1] > 1):
            return False
    for column in np.transpose(board):
        if True in (np.unique(column[column != None],return_counts=True)[1] > 1):
            return False
    for block in chunkify(board):
        if True in (np.unique(block[block != None],return_counts=True)[1] > 1):
            return False
    return True

In [3]:
def solve(board):
    for ri,row in enumerate(board):
        for vi,value in enumerate(row):
            if value is None:
                for i in range(1,len(board)+1):
                    board = np.array(board)
                    board[ri][vi] = i
                    if verify(board):
                        result = solve(board)
                        if result is not None:
                            return result
                return None
    if verify (board):
        return board
    return None

In [4]:
from playwright.async_api import async_playwright
import asyncio

async def get_board_html():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto('https://www.nytimes.com/puzzles/sudoku/hard', wait_until="networkidle")

        html = await page.content()

        await browser.close()

        return html

# Run it
html = await get_board_html()



In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

cells = soup.find_all("div", {"data-testid": "sudoku-cell-100"})
values = [cell.get("aria-label") for cell in cells]
board = []
index = 0
while len(soup.find_all("div", {"data-testid": f"sudoku-cell-{index}"})) >0:
    board.append(soup.find_all("div", {"data-testid": f"sudoku-cell-{index}"})[0].get("aria-label"))
    index += 1

In [6]:
board = np.array(board)
board = board.astype(object)
board[board == 'empty'] = None
board = np.array([int(x) if x is not None else None for x in board], dtype=object)
n = int(np.sqrt(board.size))
board = np.array(board)
board = board.reshape(n,n)
print (board)

[[1 None 5 None None None 8 None None]
 [None None None 3 None None None None None]
 [4 None 9 None None None None None 5]
 [None None 8 9 2 None 6 None None]
 [7 None None 6 None None None None None]
 [2 None 4 8 1 None None None 3]
 [None None None 2 None None None 4 None]
 [None None None None 4 None 9 7 None]
 [None None None None 9 None None None None]]


In [7]:
print(solve(board))

[[1 2 5 4 6 9 8 3 7]
 [8 7 6 3 5 1 4 2 9]
 [4 3 9 7 8 2 1 6 5]
 [3 5 8 9 2 7 6 1 4]
 [7 9 1 6 3 4 2 5 8]
 [2 6 4 8 1 5 7 9 3]
 [9 1 3 2 7 8 5 4 6]
 [6 8 2 5 4 3 9 7 1]
 [5 4 7 1 9 6 3 8 2]]


In [ ]:
import time
async def play_board(board):
    board = np.array(board).flatten()
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto('https://www.nytimes.com/puzzles/sudoku/hard', wait_until="networkidle")
        await page.get_by_role("button", name="Reject all").click()
        await page.get_by_role("button", name="Close").click()
        await page.get_by_test_id("sudoku-cell-0").click()
        for value in board:
            await page.keyboard.press(value)
            await page.locator("body").press("ArrowRight")
        time.sleep(10)
        return


await play_board(board)

Error: Keyboard.press: Unknown key: "hi"

In [ ]:
import re
from playwright.sync_api import Playwright, sync_playwright, expect


def run(playwright: Playwright) -> None:
    browser = playwright.chromium.launch(headless=False)
    context = browser.new_context()
    page = context.new_page()
    page.goto("https://www.nytimes.com/puzzles/sudoku/hard")
    page.get_by_role("button", name="Reject all").click()
    page.get_by_role("img", name="New feature preview").click()
    page.get_by_role("button", name="Close").click()
    page.get_by_test_id("sudoku-cell-0").click()
    page.locator("body").press("ArrowRight")
    page.locator("body").press("ArrowRight")

    # ---------------------
    context.close()
    browser.close()


with sync_playwright() as playwright:
    run(playwright)


TypeError: 'PlaywrightContextManager' object does not support the context manager protocol